In [1]:
import sympy as sp

In [2]:
import matplotlib.pyplot as plt

In [3]:
import sympy.physics.units.quantities as sq

In [4]:
from sympy.physics.quantum.constants import hbar

In [5]:
PSI_FUNCTION = sp.Function( "psi" )
POTENTIAL_FUNCTION = sp.Function( "V" )
TOTAL_ENERGY_SYMBOL = sq.Symbol( 'E', nonzero = True, positive = True )
MASS_SYMBOL = sq.Quantity( 'm', positive = True, nonzero = True )
#REDUCED_PLANCK_CONSTANT_SYMBOL = sq.Symbol( "hbar" )
POSITION_SYMBOL = sp.Symbol( 'x', positive = True )

def time_independent_schroedinger_equation( 
            psi = PSI_FUNCTION, 
            potential = POTENTIAL_FUNCTION, 
            total_energy = TOTAL_ENERGY_SYMBOL, 
            mass = MASS_SYMBOL, 
            reduced_planck_constant = hbar, #REDUCED_PLANCK_CONSTANT_SYMBOL, 
            position = POSITION_SYMBOL 
        ): 
    return sp.Eq( 
            ( ( -( reduced_planck_constant ** 2 ) / ( 2 * mass ) ) \
                    * sp.Derivative( psi( position ), ( position, 2 ) ) )
                    + ( potential( position ) * psi( position ) ), 
            total_energy * psi( position ) 
        )


In [6]:
set_equal = lambda to_set, value : sp.Eq( to_set, value )
both_sides = lambda equation, opreation : sp.Eq( opreation( equation.lhs ), opreation( equation.rhs ) )
equation_to_dict = lambda equation : { equation.lhs : equation.rhs }

In [7]:
hbar_eq = sp.Eq( hbar, sp.physics.units.planck / ( 2 * sp.pi ) )

In [8]:
hbar_eq

Eq(hbar, planck/(2*pi))

In [9]:
psi = PSI_FUNCTION
potential = POTENTIAL_FUNCTION
total_energy = TOTAL_ENERGY_SYMBOL
mass = MASS_SYMBOL
#reduced_planck_constant = REDUCED_PLANCK_CONSTANT_SYMBOL
x = POSITION_SYMBOL

In [10]:
k = sp.Symbol( 'k' )

In [11]:
k_squared = sp.Eq( k ** 2, ( ( 2 * total_energy * mass ) / ( hbar ** 2 ) ) )

In [12]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [13]:
class ZeroPotential( sp.Function ): 
    @classmethod
    def eval( cls, position ):
        return 0

In [14]:
#https://docs.sympy.org/latest/modules/assumptions/index.html#querying
#https://docs.sympy.org/latest/modules/codegen.html#module-sympy.codegen.cxxnodes

class Potential( sp.Function ): 
    DEFAULT_POTENTIAL = sp.Symbol( "V_0" )
    @classmethod
    def eval( cls, position, potential = DEFAULT_POTENTIAL ): 
        return potential

In [15]:
class TunnelPotential( sp.Function ): 
    DEFAULT_WELL_LENGTH = sp.Symbol( 'L' )
    DEFAULT_POTENTIAL = Potential.DEFAULT_POTENTIAL
    DEFAULT_START = 0
    @classmethod
    def eval( cls, position, length = DEFAULT_WELL_LENGTH, 
             start = DEFAULT_START, potential = DEFAULT_POTENTIAL ): 
        if position < start or position > sp.simplify( length + start ): 
            return ZeroPotential.eval( position )
        return Potential.eval( position, potential )

In [16]:
class Stepper: 

    LEFT = "LEFT", 
    RIGHT = "RIGHT"
    
    def __init__( self, first_step, new_steps = None ): 
        self.steps = new_steps if new_steps else []
        self.steps.append( first_step )
        self.chaining = False
    
    def step_number( self, step = None ): 
        return ( len( self.steps ) - 1 ) if not step else step
    
    def last_step( self, step = None ):
        return self.steps[ self.step_number( step ) ]
    
    def _return_chain( self, step, chain ): 
        return self if ( chain or self.chaining ) else step        
    
    def add_step( self, new_step, chain = False ):
        self.steps.append( new_step )
        return self._return_chain( self.steps[ -1 ], chain )

    def operate( self, operation, step = None, chain = False ):
        self.steps.append( operation( self.last_step( step ) ) )
        return self._return_chain( self.steps[ -1 ], chain )

    def manipulate( self, operation, chain = False ): 
        return self._return_chain( self.add_step( 
                both_sides( self.last_step(), operation ) ), 
                chain 
            )
    
    def delete_step( self, step ): 
        old_step = self.steps[ step ]
        del self.steps[ step ]
        return old_step
    
    def undo( self, step = None, chain = False ): 
        return self._return_chain( 
                self.delete_step( self.step_number( step ) ), 
                chain 
            )
    
    def clone( self, from_step = None ):
        return self.branch( lambda blank : blank, from_step )
    
    def branch( self, operation = None, from_step = None ): 
        from_step = self.step_number( from_step )
        return Stepper( 
                both_sides( self.last_step( from_step ), operation ), 
                self.steps[ : self.step_number( from_step ) : ], 
            )
    
    def substitute_constant( self, constant, chain = False ):
            return self.operate( lambda step : step.subs( { constant.rhs : constant.lhs } ) )
    
    def negate_add( self, side_to_negate_add, chain = False ): 
        last_step = self.last_step()
        return self.manipulate( 
                lambda side : side + -( 
                    last_step.lhs if side_to_negate_add == Stepper.LEFT 
                            else last_step.rhs 
                        ), 
                chain 
            )
    
    def rename( self, old_symbol, new_symbol_name, chain = False ): 
        last_step = self.last_step()
        assert old_symbol in last_step.atoms() or old_symbol in last_step.atoms( sp.Function )
        return self.add_step( last_step.subs( 
            old_symbol if type( old_symbol ) == dict \
                    else { old_symbol : new_symbol_name } ), chain 
                )
    
    def chain( self, operation ):
        self.chaining = True
        operation( self )
        self.chaining = False
        return self
    
    def begin_chain( self ): 
        self.chaining = True
        return self
    
    def end_chain( self ): 
        self.chaining = False
        return self


In [17]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [18]:
psi_region = [
        Stepper( time_independent_schroedinger_equation( potential = ZeroPotential ) ), 
        Stepper( time_independent_schroedinger_equation( potential = Potential ) ), 
        Stepper( time_independent_schroedinger_equation( potential = ZeroPotential ) )
    ]

In [19]:
psi_region[ 0 ].last_step()

Eq(-hbar**2*Derivative(psi(x), (x, 2))/(2*m), E*psi(x))

In [20]:
psi_region[ 0 ].manipulate( lambda side : ( side - psi_region[ 0 ].last_step().rhs ).simplify() )

Eq(-E*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [21]:
psi_region[ 0 ].manipulate( lambda side : -( side / total_energy ).simplify() )

Eq(psi(x) + hbar**2*Derivative(psi(x), (x, 2))/(2*m*E), 0)

In [22]:
psi_region[ 0 ].manipulate( lambda side : side - psi( x ) )

Eq(hbar**2*Derivative(psi(x), (x, 2))/(2*m*E), -psi(x))

In [23]:
psi_region[ 0 ].manipulate( lambda side : side * k_squared.rhs )

Eq(Derivative(psi(x), (x, 2)), -2*m*E*psi(x)/hbar**2)

In [24]:
psi_region[ 0 ].manipulate( lambda side : side / psi( x ) )

Eq(Derivative(psi(x), (x, 2))/psi(x), -2*m*E/hbar**2)

In [25]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [26]:
psi_region[ 0 ].substitute_constant( k_squared )

Eq(Derivative(psi(x), (x, 2))/psi(x), -k**2)

In [27]:
psi_region[ 0 ].manipulate( lambda side : side * psi( x ) )

Eq(Derivative(psi(x), (x, 2)), -k**2*psi(x))

In [28]:
psi_region[ 0 ].negate_add( Stepper.RIGHT )

Eq(k**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [29]:
# For later

psi_region[ 2 ] = psi_region[ 0 ].clone()

In [30]:
psi_region[ 2 ].last_step()

Eq(k**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [31]:
k_0 = sp.Symbol( "k_0" )

In [32]:
psi_region[ 0 ].rename( k, k_0 )

Eq(k_0**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [33]:
diff_sol = sp.solvers.ode.dsolve( psi_region[ 0 ].last_step().lhs, 0, ivar = x )
#ics = { psi( 0 ): 0, psi( well_length ): 0 }

In [34]:
diff_sol

Eq(psi(x), C1*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [35]:
psi_region[ 0 ].add_step( diff_sol )

Eq(psi(x), C1*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [36]:
C1 = sp.Symbol( "C1" )
C2 = sp.Symbol( "C2" )
constants = [ [ sp.Symbol( 'A' ), sp.Symbol( 'B' ) ] ]

psi_region[ 0 ].rename( C1, constants[ 0 ][ 0 ] )

Eq(psi(x), A*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [37]:
psi_region[ 0 ].rename( C2, constants[ 0 ][ 1 ] )

Eq(psi(x), A*exp(-I*k_0*x) + B*exp(I*k_0*x))

In [38]:
psi_region[ 1 ].last_step()

Eq(V_0*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), E*psi(x))

In [39]:
psi_region[ 0 ].last_step()

Eq(psi(x), A*exp(-I*k_0*x) + B*exp(I*k_0*x))

In [40]:
psi_region[ 1 ].negate_add( Stepper.RIGHT )

Eq(-E*psi(x) + V_0*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [41]:
psi_region[ 1 ].manipulate( lambda side : side.collect( psi(x) ) )

Eq((-E + V_0)*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [42]:
psi_region[ 1 ].last_step().lhs.as_two_terms()[ 1 ]

-hbar**2*Derivative(psi(x), (x, 2))/(2*m)

In [43]:
region_1_clone = psi_region[ 1 ].clone()

region_1_clone.manipulate( lambda side : side 
                - region_1_clone.last_step()
                        .lhs.as_two_terms()[ 1 ]
        )

Eq((-E + V_0)*psi(x), hbar**2*Derivative(psi(x), (x, 2))/(2*m))

In [44]:
region_1_clone.manipulate( lambda side : side / region_1_clone.last_step().lhs.as_two_terms()[ 0 ] )

Eq(psi(x), hbar**2*Derivative(psi(x), (x, 2))/(2*m*(-E + V_0)))

In [45]:
region_1_clone.manipulate( lambda side : side / region_1_clone.last_step().rhs )

Eq(2*m*(-E + V_0)*psi(x)/(hbar**2*Derivative(psi(x), (x, 2))), 1)

In [46]:
region_1_clone.manipulate( lambda side : side * sp.Derivative( psi( x ), ( x, 2 ) ) )

Eq(2*m*(-E + V_0)*psi(x)/hbar**2, Derivative(psi(x), (x, 2)))

In [47]:
region_1_clone.manipulate( lambda side : side / psi( x ) )

Eq(2*m*(-E + V_0)/hbar**2, Derivative(psi(x), (x, 2))/psi(x))

In [48]:
k_1 = sp.Symbol( "k_1" )
k_1_squared = sp.Eq( k_1 ** 2, region_1_clone.last_step().lhs )

In [49]:
k_1_squared

Eq(k_1**2, 2*m*(-E + V_0)/hbar**2)

In [50]:
region_1_clone.manipulate( lambda side : side.subs( { k_1_squared.rhs : k_1_squared.lhs } ) )

Eq(k_1**2, Derivative(psi(x), (x, 2))/psi(x))

In [51]:
region_1_clone.manipulate( lambda side : side * psi( x ) )

Eq(k_1**2*psi(x), Derivative(psi(x), (x, 2)))

In [52]:
region_1_clone.negate_add( Stepper.RIGHT )

Eq(k_1**2*psi(x) - Derivative(psi(x), (x, 2)), 0)

In [53]:
psi_region[ 1 ] = region_1_clone

In [54]:
psi_region[ 1 ].last_step()

Eq(k_1**2*psi(x) - Derivative(psi(x), (x, 2)), 0)

In [55]:
psi_1 = sp.Function( "psi_1" )

In [56]:
psi_2 = sp.Function( "psi_2" )

In [57]:
psi_region[ 1 ].rename( psi( x ), psi_1( x ) )

Eq(k_1**2*psi_1(x) - Derivative(psi_1(x), (x, 2)), 0)

In [58]:
#JUST NOTICED THAT THE SOLUTION DOES NOT CONTAINER IMAGINARY NUMBERS? WHY?

psi_region[ 1 ].add_step( sp.solvers.ode.dsolve( psi_region[ 1 ].last_step() ) )

Eq(psi_1(x), C1*exp(-k_1*x) + C2*exp(k_1*x))

In [59]:
constants.append( [ sp.Symbol( 'C' ), sp.Symbol( 'D' ) ] )

In [60]:
psi_region[ 1 ].rename( C1, constants[ 1 ][ 0 ] )

Eq(psi_1(x), C*exp(-k_1*x) + C2*exp(k_1*x))

In [61]:
psi_region[ 1 ].rename( C2, constants[ 1 ][ 1 ] )

Eq(psi_1(x), C*exp(-k_1*x) + D*exp(k_1*x))

In [62]:
k_2 = sp.Symbol( "k_2" )
psi_2_symbol = sp.Function( "psi_2" )

psi_region[ 2 ].rename( psi( x ), psi_2_symbol( x ) )

Eq(k**2*psi_2(x) + Derivative(psi_2(x), (x, 2)), 0)

In [63]:
psi_region[ 2 ].rename( k, k_2 )

Eq(k_2**2*psi_2(x) + Derivative(psi_2(x), (x, 2)), 0)

In [64]:
psi_region[ 2 ].add_step( sp.solvers.ode.dsolve( psi_region[ 2 ].last_step() ) )

Eq(psi_2(x), C1*exp(-I*k_2*x) + C2*exp(I*k_2*x))

In [65]:
constants.append( [ sp.Symbol( 'G' ) ] ) #Skipping a letter in the alphabet so there is no confusion with energy

psi_region[ 2 ].rename( C2, constants[ 2 ][ 0 ] )

Eq(psi_2(x), C1*exp(-I*k_2*x) + G*exp(I*k_2*x))

In [66]:
psi_region[ 2 ].rename( C1, 0 )

Eq(psi_2(x), G*exp(I*k_2*x))

In [67]:
#diff_sol = sp.solvers.ode.dsolve( psi_region[ 1 ].last_step().lhs, 0, ivar = x, 
#        ics = { 
#                psi_1( 0 ) : psi( 0 ), 
#                psi_1( barrier_length ) : psi_2( barrier_length ), 
#                psi_1( x ).diff( x, 1 ).subs( x, 0 ) : psi( x ).diff( x, 1 ).subs( x, 0 ), 
#                psi_1( x ).diff( x, 1 ).subs( x, barrier_length ) : psi_2( x ).diff( x, 1 ).subs( x, barrier_length )
#        }
#    )

In [68]:
flat_constants = []
for constant_set in constants: 
    flat_constants += constant_set

normalize = lambda expression, from_, to : sp.Eq( sp.integrate( expression * sp.conjugate( expression ), ( x, from_, to ) ), 1, evaluate = False )

constants_helpers = sp.solve( 
        [ equation.last_step() for equation in psi_region ], 
        flat_constants
    )

In [69]:
region = []
region.append( Stepper( sp.Eq( psi_region[ 2 ].last_step().rhs, psi_region[ 1 ].last_step().rhs ) ) )

In [70]:
region[ 0 ].rename( x, TunnelPotential.DEFAULT_WELL_LENGTH )

Eq(G*exp(I*L*k_2), C*exp(-L*k_1) + D*exp(L*k_1))

In [71]:
region.append( Stepper( sp.Eq( psi_region[ 0 ].last_step().rhs, psi_region[ 1 ].last_step().rhs ) ) )

In [72]:
region[ 1 ].rename( x, 0 )

Eq(A + B, C + D)

In [73]:
region.append( Stepper( sp.Eq( sp.diff( psi_region[ 0 ].last_step().rhs, x ), sp.diff( psi_region[ 1 ].last_step().rhs, x ) ) ) )

In [74]:
region[ 2 ].rename( x, 0 )

Eq(-I*A*k_0 + I*B*k_0, -C*k_1 + D*k_1)

In [75]:
region.append( Stepper( sp.Eq( sp.diff( psi_region[ 2 ].last_step().rhs, x ), sp.diff( psi_region[ 1 ].last_step().rhs, x ) ) ) )

In [76]:
region[ 3 ].rename( x, TunnelPotential.DEFAULT_WELL_LENGTH )

Eq(I*G*k_2*exp(I*L*k_2), -C*k_1*exp(-L*k_1) + D*k_1*exp(L*k_1))

In [77]:
constants_system = [ equation.last_step() for equation in region ]

In [78]:
constants_system

[Eq(G*exp(I*L*k_2), C*exp(-L*k_1) + D*exp(L*k_1)),
 Eq(A + B, C + D),
 Eq(-I*A*k_0 + I*B*k_0, -C*k_1 + D*k_1),
 Eq(I*G*k_2*exp(I*L*k_2), -C*k_1*exp(-L*k_1) + D*k_1*exp(L*k_1))]

In [79]:
constants_solutions = sp.solve( constants_system )

In [80]:
constants_solutions = constants_solutions[ 0 ]
constants_solutions

{A: G*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1),
 B: G*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(-k_1 - I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1),
 C: G*(k_1 - I*k_2)*exp(L*(k_1 + I*k_2))/(2*k_1),
 D: G*(k_1 + I*k_2)*exp(-L*(k_1 - I*k_2))/(2*k_1)}

In [81]:
#solve_g = region[ 3 ].clone()
#solve_g.rename( constants[ 1 ][ 0 ], constants_solutions[ constants[ 1 ][ 0 ] ] )
#solve_g.rename( constants[ 1 ][ 1 ], constants_solutions[ constants[ 1 ][ 1 ] ] )
#solve_g.manipulate( lambda side : side * 2 )
#solve_g.manipulate( lambda side : side / sp.exp( sp.I * TunnelPotential.DEFAULT_WELL_LENGTH * k_2 ) )
#solve_g.add_step( sp.Eq( solve_g.last_step().lhs, solve_g.last_step().rhs.collect( sp.Symbol( 'G' ) ) ) )
#solve_g.negate_add( Stepper.RIGHT )
#solve_g.add_step( sp.Eq( solve_g.last_step().lhs.collect( sp.Symbol( 'G' ) ), solve_g.last_step().rhs ) )

In [82]:
G = sp.Symbol( 'G', real = True )

In [132]:
g_normalized = Stepper( normalize( psi_region[ 2 ].last_step().rhs, 0, TunnelPotential.DEFAULT_WELL_LENGTH ) )

In [142]:
g_normalized.last_step()

Eq(I*G*exp(I*L*k_2)*conjugate(G)/(-k_2*exp(I*L*conjugate(k_2)) + exp(I*L*conjugate(k_2))*conjugate(k_2)) - I*G*conjugate(G)/(-k_2 + conjugate(k_2)), 1)

In [143]:
normalize_no_conjugate = lambda expression, from_, to : sp.Eq( sp.integrate( expression ** 2, ( x, from_, to ) ), 1, evaluate = False )

In [144]:
g_normalized_no_conjugate = Stepper( normalize_no_conjugate( psi_region[ 2 ].last_step().rhs, 0, TunnelPotential.DEFAULT_WELL_LENGTH ) )

In [159]:
g_solver = Stepper( sp.Eq( g_normalized_no_conjugate.last_step().lhs.args[ 0 ][ 0 ], 1 ) )

In [160]:
g_solver.last_step()

Eq(-I*G**2*exp(2*I*L*k_2)/(2*k_2) + I*G**2/(2*k_2), 1)

In [166]:
g_solver.manipulate( lambda side : side * 2 * k_2 / G**2 )

Eq(k_2*(-I*G**2*exp(2*I*L*k_2)/k_2 + I*G**2/k_2)/G**2, 2*k_2/G**2)

In [179]:
g_solver.manipulate( lambda side : side.cancel().simplify() )

Eq(I*G**2*(1 - exp(2*I*L*k_2))/G**2, 2*k_2/G**2)

In [213]:
g_solver_ordered_factors = g_solver.last_step().lhs.as_ordered_factors()

In [220]:
g_solver_factor = g_solver_ordered_factors[ 0 ] * g_solver_ordered_factors[ 3 ]

In [223]:
g_solver_factor

I*(1 - exp(2*I*L*k_2))

In [221]:
g_solver_numer_denom = g_solver.last_step().rhs.as_numer_denom()

In [225]:
sp.Eq( G**2, g_solver_numer_denom[ 0 ] / g_solver_factor )

Eq(G**2, -2*I*k_2/(1 - exp(2*I*L*k_2)))

In [230]:
g_solver.manipulate( lambda side : side / g_solver_factor )

Eq(G**2/G**2, -2*I*k_2/(G**2*(1 - exp(2*I*L*k_2))))

In [235]:
g_solver.manipulate( lambda side : side.subs( { g_solver.last_step().lhs : 1 } ) )

Eq(1, -2*I*k_2/(G**2*(1 - exp(2*I*L*k_2))))

In [236]:
g_solver.manipulate( lambda side : side * G ** 2  )

Eq(G**2, -2*I*k_2/(1 - exp(2*I*L*k_2)))

In [240]:
full_constant_solutions

{A: G*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1),
 B: G*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(-k_1 - I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(4*k_0*k_1),
 C: G*(k_1 - I*k_2)*exp(L*(k_1 + I*k_2))/(2*k_1),
 D: G*(k_1 + I*k_2)*exp(-L*(k_1 - I*k_2))/(2*k_1)}

In [276]:
A = sp.Symbol( 'A' )
B = sp.Symbol( 'B' )
C = sp.Symbol( 'C' )
D = sp.Symbol( 'D' )

In [277]:
GG = list( constants_solutions[ A ].atoms() )[ 1 ]
constants_solutions[ A ].subs( { GG : g_solver.last_step().rhs } )

-I*k_2*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 + (-k_1 + I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(2*k_0*k_1*(1 - exp(2*I*L*k_2)))

In [278]:
full_constant_solutions = { current_constant : constants_solutions[ current_constant ].subs( { GG : g_solver.last_step().rhs } ).simplify() for current_constant in constants_solutions }

In [279]:
full_constant_solutions

{A: I*k_2*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 - (k_1 - I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(2*k_0*k_1*(exp(2*I*L*k_2) - 1)),
 B: I*k_2*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) - I*k_1*(k_1 + I*k_2 - (k_1 - I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(2*k_0*k_1*(exp(2*I*L*k_2) - 1)),
 C: I*k_2*(k_1 - I*k_2)*exp(L*(k_1 + I*k_2))/(k_1*(exp(2*I*L*k_2) - 1)),
 D: I*k_2*(k_1 + I*k_2)*exp(-L*(k_1 - I*k_2))/(k_1*(exp(2*I*L*k_2) - 1))}

In [280]:
full_constant_solutions[ A ]

I*k_2*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) + I*k_1*(k_1 + I*k_2 - (k_1 - I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(2*k_0*k_1*(exp(2*I*L*k_2) - 1))

In [281]:
full_constant_solutions[ B ]

I*k_2*(k_0*(k_1 + I*k_2 + (k_1 - I*k_2)*exp(2*L*k_1)) - I*k_1*(k_1 + I*k_2 - (k_1 - I*k_2)*exp(2*L*k_1)))*exp(-L*(k_1 - I*k_2))/(2*k_0*k_1*(exp(2*I*L*k_2) - 1))

In [282]:
full_constant_solutions[ C ]

I*k_2*(k_1 - I*k_2)*exp(L*(k_1 + I*k_2))/(k_1*(exp(2*I*L*k_2) - 1))

In [283]:
full_constant_solutions[ D ]

I*k_2*(k_1 + I*k_2)*exp(-L*(k_1 - I*k_2))/(k_1*(exp(2*I*L*k_2) - 1))

In [405]:
q = psi_region[ 0 ].last_step().subs( full_constant_solutions ).simplify().rhs.cancel().cancel().refine().collect( k_2 ).collect( k_1 ).collect( k_0 )

In [406]:
q

(k_2**2*(k_0*(exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x) + exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2)*exp(2*I*k_0*x) - exp(I*L*k_2)) + k_1*(I*exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x) - I*exp(2*L*k_1)*exp(I*L*k_2) + I*exp(I*L*k_2)*exp(2*I*k_0*x) - I*exp(I*L*k_2))) + k_2*(k_0*k_1*(I*exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x) + I*exp(2*L*k_1)*exp(I*L*k_2) + I*exp(I*L*k_2)*exp(2*I*k_0*x) + I*exp(I*L*k_2)) + k_1**2*(-exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x) + exp(2*L*k_1)*exp(I*L*k_2) + exp(I*L*k_2)*exp(2*I*k_0*x) - exp(I*L*k_2))))/(k_0*k_1*(2*exp(L*k_1)*exp(2*I*L*k_2)*exp(I*k_0*x) - 2*exp(L*k_1)*exp(I*k_0*x)))

In [407]:
qq = ( q.as_numer_denom()[ 1 ].as_ordered_factors()[ 2 ] / 2 ).as_two_terms()

In [408]:
factors_qq = ( q.as_numer_denom()[ 0 ].as_ordered_terms()[ 0 ] / ( k_2 ** 2 ) ).as_ordered_terms()[ 0 ].as_ordered_factors()[ 1 ]

In [409]:
factors_qq

exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x) + exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2)*exp(2*I*k_0*x) - exp(I*L*k_2)

In [521]:
to_replace = [ i[ 0 ] for i in factors_qq.as_terms()[ 0 ] ]

In [522]:
to_replace

[-exp(I*L*k_2),
 exp(2*L*k_1)*exp(I*L*k_2),
 -exp(I*L*k_2)*exp(2*I*k_0*x),
 exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x)]

In [523]:
sub_table = { to_replace[ i ] : sp.Symbol( "M_" + str( i ) ) for i in range( len( to_replace ) ) }

In [524]:
sub_table

{-exp(I*L*k_2): M_0,
 exp(2*L*k_1)*exp(I*L*k_2): M_1,
 -exp(I*L*k_2)*exp(2*I*k_0*x): M_2,
 exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x): M_3}

In [525]:
sub_table_invert = { -key : -sub_table[ key ] for key in sub_table }

In [526]:
sub_table_invert

{exp(I*L*k_2): -M_0,
 -exp(2*L*k_1)*exp(I*L*k_2): -M_1,
 exp(I*L*k_2)*exp(2*I*k_0*x): -M_2,
 -exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x): -M_3}

In [527]:
sub_table_i = { -sp.I * key : -sp.I * sub_table[ key ] for key in sub_table }

In [528]:
sub_table_i

{I*exp(I*L*k_2): -I*M_0,
 -I*exp(2*L*k_1)*exp(I*L*k_2): -I*M_1,
 I*exp(I*L*k_2)*exp(2*I*k_0*x): -I*M_2,
 -I*exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x): -I*M_3}

In [529]:
e_ik_0x = psi_region[ 0 ].last_step().rhs.as_ordered_terms()[ 1 ].as_ordered_terms()[ 0 ].as_two_terms()[ 1 ]

In [530]:
e_ik_0x

exp(I*k_0*x)

In [531]:
sub_table_e_ik_0x = { key * e_ik_0x : e_ik_0x * sub_table[ key ]  for key in sub_table }

In [532]:
sub_table_e_ik_0x

{-exp(I*L*k_2)*exp(I*k_0*x): M_0*exp(I*k_0*x),
 exp(2*L*k_1)*exp(I*L*k_2)*exp(I*k_0*x): M_1*exp(I*k_0*x),
 -exp(I*L*k_2)*exp(3*I*k_0*x): M_2*exp(I*k_0*x),
 exp(2*L*k_1)*exp(I*L*k_2)*exp(3*I*k_0*x): M_3*exp(I*k_0*x)}

In [533]:
e_iLk_1 = e_ik_0x.subs( { k_0 : k_1, x : TunnelPotential.DEFAULT_WELL_LENGTH  } )

In [534]:
e_Lk_1 = e_iLk_1.subs( { sp.I : 1 } )

In [535]:
e_Lk_2 = e_Lk_1.subs( { k_1 : k_2 } )

In [536]:
e_2iLk_1 = e_iLk_1.subs( { k_1 : k_2 } )

In [537]:
full_sub_table = { 
    **sub_table, 
    **sub_table_invert, 
    **sub_table_i, 
    **sub_table_e_ik_0x, 
    e_ik_0x : sp.Symbol( "M_3" ), 
    e_iLk_1 : sp.Symbol( "M_4" ), 
    e_Lk_1 : sp.Symbol( "M_5" ), 
    e_Lk_2 : sp.Symbol( "M_6" ), 
    e_2iLk_1 : sp.Symbol( "M_7" )
}

In [538]:
full_sub_table

{-exp(I*L*k_2): M_0,
 exp(2*L*k_1)*exp(I*L*k_2): M_1,
 -exp(I*L*k_2)*exp(2*I*k_0*x): M_2,
 exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x): M_3,
 exp(I*L*k_2): M_7,
 -exp(2*L*k_1)*exp(I*L*k_2): -M_1,
 exp(I*L*k_2)*exp(2*I*k_0*x): -M_2,
 -exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x): -M_3,
 I*exp(I*L*k_2): -I*M_0,
 -I*exp(2*L*k_1)*exp(I*L*k_2): -I*M_1,
 I*exp(I*L*k_2)*exp(2*I*k_0*x): -I*M_2,
 -I*exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x): -I*M_3,
 -exp(I*L*k_2)*exp(I*k_0*x): M_0*exp(I*k_0*x),
 exp(2*L*k_1)*exp(I*L*k_2)*exp(I*k_0*x): M_1*exp(I*k_0*x),
 -exp(I*L*k_2)*exp(3*I*k_0*x): M_2*exp(I*k_0*x),
 exp(2*L*k_1)*exp(I*L*k_2)*exp(3*I*k_0*x): M_3*exp(I*k_0*x),
 exp(I*k_0*x): M_3,
 exp(I*L*k_1): M_4,
 exp(L*k_1): M_5,
 exp(L*k_2): M_6}

In [539]:
#qqq = q.subs( sub_table ) \
#        .subs( sub_table_invert ) \
#        .subs( sub_table_i ) \
#        .subs( sub_table_e_ik_0x ) \
#        .subs( { e_ik_0x : sp.Symbol( "M_3" ) } ) \
#        .subs( { e_iLk_1 : sp.Symbol( "M_4" ) } ) \
#        .subs( { e_Lk_1 : sp.Symbol( "M_5" ) } ) \
#        .subs( { e_Lk_2 : sp.Symbol( "M_6" ) } ) \
#        .subs( { e_2iLk_1 : sp.Symbol( "M_7" ) } )
qqq = q.subs( full_sub_table )

In [540]:
qqq

(k_2**2*(k_0*(M_0 + M_1 + M_2 + M_3) + k_1*(I*M_0 - I*M_1 - I*M_2 + I*M_3)) + k_2*(k_0*k_1*(-I*M_0 + I*M_1 - I*M_2 + I*M_3) + k_1**2*(M_0 + M_1 - M_2 - M_3)))/(k_0*k_1*(2*M_3*M_5*M_7**2 - 2*M_3*M_5))

In [541]:
qqqq = [ qqq.simplify().collect( "M_" + str( i ) ) for i in range( 0, 7 ) ][ -1 ]

In [543]:
qqqq.collect( k_2 ).collect( k_1 ).collect( k_0 ).cancel().collect( k_0 ).collect( k_1 ).collect( k_2 ).simplify().refine()

k_2*(-k_0*(I*k_1*(M_0 - M_1 + M_2 - M_3) - k_2*(M_0 + M_1 + M_2 + M_3)) + k_1**2*(M_0 + M_1 - M_2 - M_3) + I*k_1*k_2*(M_0 - M_1 - M_2 + M_3))/(2*M_3*M_5*k_0*k_1*(M_7**2 - 1))

In [554]:
inverse_sub_table = { full_sub_table[ key ] : key for key in full_sub_table }

In [568]:
qqqq_ = qqqq.subs( inverse_sub_table )

In [569]:
qqqq_

k_2*(-k_1*(I*k_0*(-exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x) - exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2)*exp(2*I*k_0*x) - exp(I*L*k_2)) + k_1*(exp(2*L*k_1)*exp(I*L*k_2)*exp(2*I*k_0*x) - exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2)*exp(2*I*k_0*x) + exp(I*L*k_2))) + k_2*(k_0*(exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2)*exp(2*I*k_0*x) - exp(I*L*k_2) + exp(I*k_0*x)) + I*k_1*(-exp(2*L*k_1)*exp(I*L*k_2) + exp(I*L*k_2)*exp(2*I*k_0*x) - exp(I*L*k_2) + exp(I*k_0*x))))*exp(-L*k_1)*exp(-I*k_0*x)/(2*k_0*k_1*(exp(2*I*L*k_2) - 1))

In [573]:
psi_0 = sp.lambdify( x, qqqq_, 'numpy' )

In [563]:
from sympy.plotting import plot

In [580]:
from matplotlib import pyplot as plt

In [ ]:
#x_axis = np.linspace( -1, 0 )
plt.plot( x_axis, psi_0( x_axis ) )

In [593]:
qqqq_.collect( e_ik_0x ).collect( e_ik_0x )

k_2*(-k_1*(I*k_0*((-exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2))*exp(2*I*k_0*x) - exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2)) + k_1*((exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2))*exp(2*I*k_0*x) - exp(2*L*k_1)*exp(I*L*k_2) + exp(I*L*k_2))) + k_2*(k_0*(exp(2*L*k_1)*exp(I*L*k_2) - exp(I*L*k_2)*exp(2*I*k_0*x) - exp(I*L*k_2) + exp(I*k_0*x)) + I*k_1*(-exp(2*L*k_1)*exp(I*L*k_2) + exp(I*L*k_2)*exp(2*I*k_0*x) - exp(I*L*k_2) + exp(I*k_0*x))))*exp(-L*k_1)*exp(-I*k_0*x)/(2*k_0*k_1*(exp(2*I*L*k_2) - 1))

In [601]:
k_0_eqn = sp.Eq( k_0, sp.sqrt( k_squared.subs( { k : k_0 } ).rhs ) )

In [602]:
k_0_eqn

Eq(k_0, sqrt(2)*sqrt(m)*sqrt(E)/hbar)

In [603]:
k_2_eqn = sp.Eq( k_2, sp.sqrt( k_squared.subs( { k : k_2 } ).rhs ) )

In [604]:
k_2_eqn

Eq(k_2, sqrt(2)*sqrt(m)*sqrt(E)/hbar)

In [610]:
k_1_eqn = sp.Eq( k_1, sp.sqrt( k_1_squared.rhs ) )

In [611]:
k_1_eqn

Eq(k_1, sqrt(2)*sqrt(m)*sqrt(-E + V_0)/hbar)

In [620]:
#k_1_eqn.lhs : k_1_eqn.rhs,
qq_qq = qqqq_.subs( { k_0_eqn.lhs : k_0_eqn.rhs,  k_2_eqn.lhs : k_2_eqn.rhs } ).simplify().refine().trigsimp().subs( { k_1_eqn.lhs : k_1_eqn.rhs } ).simplify().refine().trigsimp()

In [621]:
qq_qq

sqrt(2)*sqrt(m)*(I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*sqrt(m)*(I*sqrt(E)*L + 2*I*sqrt(E)*x + 2*L*sqrt(-E + V_0))/hbar) + I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*x/hbar) + 2*I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar) + E*exp(sqrt(2)*sqrt(m)*L*(I*sqrt(E) + 2*sqrt(-E + V_0))/hbar) - E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) + E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*x/hbar) - E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar) + (E - V_0)*exp(sqrt(2)*sqrt(m)*(I*sqrt(E)*L + 2*I*sqrt(E)*x + 2*L*sqrt(-E + V_0))/hbar) - (E - V_0)*exp(sqrt(2)*sqrt(m)*L*(I*sqrt(E) + 2*sqrt(-E + V_0))/hbar) + (E - V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) - (E - V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar))*exp(-sqrt(2)*sqrt(m)*(I*sqrt(E)*x + L*sqrt(-E + V_0))/hbar)/(2*hbar*sqrt(-E + V_0)*(exp(2*sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) - 1))

In [626]:
qq_qq.collect( k_0_eqn.rhs ).simplify()

sqrt(2)*sqrt(m)*(I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*sqrt(m)*(I*sqrt(E)*L + 2*I*sqrt(E)*x + 2*L*sqrt(-E + V_0))/hbar) + I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*x/hbar) + 2*I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar) + E*exp(sqrt(2)*sqrt(m)*L*(I*sqrt(E) + 2*sqrt(-E + V_0))/hbar) - E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) + E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*x/hbar) - E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar) + (E - V_0)*exp(sqrt(2)*sqrt(m)*(I*sqrt(E)*L + 2*I*sqrt(E)*x + 2*L*sqrt(-E + V_0))/hbar) - (E - V_0)*exp(sqrt(2)*sqrt(m)*L*(I*sqrt(E) + 2*sqrt(-E + V_0))/hbar) + (E - V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) - (E - V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar))*exp(-sqrt(2)*sqrt(m)*(I*sqrt(E)*x + L*sqrt(-E + V_0))/hbar)/(2*hbar*sqrt(-E + V_0)*(exp(2*sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) - 1))

In [630]:
q2 = qqqq_.simplify().cancel().subs( { k_0_eqn.lhs : k_0_eqn.rhs,  k_2_eqn.lhs : k_2_eqn.rhs } ).simplify().refine().trigsimp().subs( { k_1_eqn.lhs : k_1_eqn.rhs } ).simplify().refine().trigsimp()

In [631]:
q2

sqrt(2)*sqrt(m)*(I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*sqrt(m)*(I*sqrt(E)*L + 2*I*sqrt(E)*x + 2*L*sqrt(-E + V_0))/hbar) + I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*x/hbar) + 2*I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar) + E*exp(sqrt(2)*sqrt(m)*L*(I*sqrt(E) + 2*sqrt(-E + V_0))/hbar) - E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) + E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*x/hbar) - E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar) + (E - V_0)*exp(sqrt(2)*sqrt(m)*(I*sqrt(E)*L + 2*I*sqrt(E)*x + 2*L*sqrt(-E + V_0))/hbar) - (E - V_0)*exp(sqrt(2)*sqrt(m)*L*(I*sqrt(E) + 2*sqrt(-E + V_0))/hbar) + (E - V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) - (E - V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar))*exp(-sqrt(2)*sqrt(m)*(I*sqrt(E)*x + L*sqrt(-E + V_0))/hbar)/(2*hbar*sqrt(-E + V_0)*(exp(2*sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) - 1))

In [640]:
q2.as_ordered_factors()[ 6]

I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*sqrt(m)*(I*sqrt(E)*L + 2*I*sqrt(E)*x + 2*L*sqrt(-E + V_0))/hbar) + I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*x/hbar) + 2*I*sqrt(E)*sqrt(-E + V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar) + E*exp(sqrt(2)*sqrt(m)*L*(I*sqrt(E) + 2*sqrt(-E + V_0))/hbar) - E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) + E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*x/hbar) - E*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar) + (E - V_0)*exp(sqrt(2)*sqrt(m)*(I*sqrt(E)*L + 2*I*sqrt(E)*x + 2*L*sqrt(-E + V_0))/hbar) - (E - V_0)*exp(sqrt(2)*sqrt(m)*L*(I*sqrt(E) + 2*sqrt(-E + V_0))/hbar) + (E - V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*L/hbar) - (E - V_0)*exp(sqrt(2)*I*sqrt(m)*sqrt(E)*(L + 2*x)/hbar)

In [650]:
psi_region[ 0 ].last_step( -3 ).rhs.trigsimp()

C1*exp(-I*k_0*x) + C2*exp(I*k_0*x)